# Reprojecting geographic data {#sec-reproj-geo-data}

## Prerequisites

Let's import the required packages:


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry
import geopandas as gpd
import rasterio
import rasterio.warp
from rasterio.plot import show
import pyproj

In [ ]:
#| echo: false
import pandas as pd
pd.options.display.max_rows = 6
pd.options.display.max_columns = 6
pd.options.display.max_colwidth = 35
plt.rcParams['figure.figsize'] = (5, 5)

and load the sample data:


In [ ]:
src_srtm = rasterio.open('data/srtm.tif')
src_nlcd = rasterio.open('data/nlcd.tif')
zion = gpd.read_file('data/zion.gpkg')
world = gpd.read_file('data/world.gpkg')

## Introduction

@sec-coordinate-reference-systems introduced coordinate reference systems (CRSs), with a focus on the two major types: geographic ('lon/lat', with units in degrees longitude and latitude) and projected (typically with units of meters from a datum) coordinate systems. This chapter builds on that knowledge and goes further. It demonstrates how to set and transform geographic data from one CRS to another and, furthermore, highlights specific issues that can arise due to ignoring CRSs that you should be aware of, especially if your data is stored with lon/lat coordinates. 

In many projects there is no need to worry about, let alone convert between, different CRSs. It is important to know if your data is in a projected or geographic coordinate system, and the consequences of this for geometry operations. However, if you know the CRS of your data and the consequences for geometry operations (covered in the next section), CRSs should just work behind the scenes: people often suddenly need to learn about CRSs when things go wrong. Having a clearly defined project CRS that all project data is in, plus understanding how and why to use different CRSs, can ensure that things don't go wrong. Furthermore, learning about coordinate systems will deepen your knowledge of geographic datasets and how to use them effectively.

This chapter teaches the fundamentals of CRSs, demonstrates the consequences of using different CRSs (including what can go wrong), and how to 'reproject' datasets from one coordinate system to another. In the next section we introduce CRSs in Python, followed by @sec-querying-and-setting-coordinate-systems which shows how to get and set CRSs associated with spatial objects. Section @sec-geometry-operations-on-projected-and-unprojected-data demonstrates the importance of knowing what CRS your data is in with reference to a worked example of creating buffers. We tackle questions of when to reproject and which CRS to use in Section @sec-when-to-reproject and Section @sec-which-crs-to-use, respectively. We cover reprojecting vector and raster objects in sections @sec-reprojecting-vector-geometries and @sec-reprojecting-raster-geometries and modifying map projections in @sec-custom-map-projections.

## Coordinate Reference Systems {#sec-coordinate-reference-systems}

Most modern geographic tools that require CRS conversions, including Python packages and desktop GIS software such as QGIS, interface with [PROJ](https://proj.org/), an open source C++ library that "transforms coordinates from one coordinate reference system (CRS) to another". CRSs can be described in many ways, including the following.

* Simple yet potentially ambiguous statements such as "it's in lon/lat coordinates".
* Formalized yet now outdated 'proj4 strings' such as `+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs`.
* With an identifying 'authority:code' text string such as `EPSG:4326`.

Each refers to the same thing: the 'WGS84' coordinate system that forms the basis of Global Positioning System (GPS) coordinates and many other datasets. But which one is correct?

The short answer is that the third way to identify CRSs is correct: `EPSG:4326` is understood by `geopandas` and `rasterio` packages covered in this book, plus many other software projects for working with geographic data including [QGIS](https://docs.qgis.org/3.22/en/docs/user_manual/working_with_projections/working_with_projections.html) and [PROJ](https://proj.org/development/quickstart.html). `EPSG:4326` is future-proof. Furthermore, although it is machine readable, unlike the proj-string representation "EPSG:4326" is short, easy to remember and highly 'findable' online (searching for EPSG:4326 yields a dedicated page on the website [epsg.io](https://epsg.io/4326), for example). The more concise identifier `4326` is also understood by `geopandas` and `rasterio`, but we recommend the more explicit `AUTHORITY:CODE` representation to prevent ambiguity and to provide context.

The longer answer is that none of the three descriptions are sufficient and more detail is needed for unambiguous CRS handling and transformations: due to the complexity of CRSs, it is not possible to capture all relevant information about them in such short text strings. For this reason, the Open Geospatial Consortium (OGC, which also developed the simple features specification that the sf package implements) developed an open standard format for describing CRSs that is called WKT (Well Known Text). This is detailed in a [100+ page document](file:///home/michael/Downloads/18-010r7_Geographic_information__Well-known_text_representation_of_coordinate_reference_systems.pdf) that "defines the structure and content of a text string implementation of the abstract model for coordinate reference systems described in ISO 19111:2019" (Open Geospatial Consortium 2019...to add citation!). The WKT representation of the WGS84 CRS, which has the identifier `EPSG:4326` is as follows:


In [ ]:
crs = pyproj.CRS.from_string('EPSG:4326') # or '.from_epsg(4326)'
print(crs.to_wkt(pretty=True))

The output of the command shows how the CRS identifier (also known as a Spatial Reference Identifier or [SRID](https://postgis.net/workshops/postgis-intro/projection.html)) works: it is simply a look-up, providing a unique identifier associated with a more complete WKT representation of the CRS. This raises the question: what happens if there is a mismatch between the identifier and the longer WKT representation of a CRS? On this point Open Geospatial Consortium (2019... to add citation!) is clear, the verbose WKT representation takes precedence over the [identifier](https://docs.opengeospatial.org/is/18-010r7/18-010r7.html#37):

> Should any attributes or values given in the cited identifier be in conflict with attributes or values given explicitly in the WKT description, the WKT values shall prevail.

The convention of referring to CRSs identifiers in the form AUTHORITY:CODE, which is also used by geographic software written in other languages, allows a wide range of formally defined coordinate systems to be referred to.26 The most commonly used authority in CRS identifiers is EPSG, an acronym for the European Petroleum Survey Group which published a standardized list of CRSs (the EPSG was [taken over](http://wiki.gis.com/wiki/index.php/European_Petroleum_Survey_Group) by the oil and gas body the Geomatics Committee of the International Association of Oil & Gas Producers (...to add citation!) in 2005). Other authorities can be used in CRS identifiers. ESRI:54030, for example, refers to ESRI’s implementation of the Robinson projection, which has the following WKT string:


In [ ]:
crs = pyproj.CRS.from_string('ESRI:54030')
print(crs.to_wkt(pretty=True))

WKT strings are exhaustive, detailed, and precise, allowing for unambiguous CRSs storage and transformations. They contain all relevant information about any given CRS, including its datum and ellipsoid, prime meridian, projection, and units.

Recent PROJ versions (6+) still allow use of proj-strings to define coordinate operations, but some proj-string keys (`+nadgrids`, `+towgs84`, `+k`, `+init=epsg:`) are either no longer supported or are discouraged. Additionally, only three datums (i.e., WGS84, NAD83, and NAD27) can be directly set in proj-string. Longer explanations of the evolution of CRS definitions and the PROJ library can be found in Bivand (2021), Chapter 2 of Pebesma and Bivand (2022), and a blog post by Floris Vanderhaeghe (...to add citations!). As outlined in the [PROJ documentation](https://proj.org/development/reference/cpp/cpp_general.html) there are different versions of the WKT CRS format including WKT1 and two variants of WKT2, the latter of which (WKT2, 2018 specification) corresponds to the ISO 19111:2019 (Open Geospatial Consortium 2019...to add citations!).

## Querying and setting coordinate systems {#sec-querying-and-setting-coordinate-systems}

Let's look at how CRSs are stored in Python spatial objects and how they can be queried and set. First we will look at getting and setting CRSs in vector geographic data objects. Consider the `GeoDataFrame` object named `world`, imported from a file `world.gpkg`. The object `world` represents countries worldwide. Its CRS can be retrieved using the `.crs` property:


In [ ]:
world.crs

The output specifies the following pieces of information:

1. The CRS type (`Geographic 2D CRS`) and EPSG code (`EPSG:4326`) 
1. The CRS name (`WGS 84`)
1. The axes (`latitude`, `longitude`) and their units (`degree`)
1. The applicable area name (`World`) and bounding box (`(-180.0, -90.0, 180.0, 90.0)`)
1. The datum (`WGS 84`)

The WKT representation, which is internally used when saving the object to a file or doing any coordinate operations, can be extracted using `.crs.to_wkt()` as shown above (@sec-coordinate-reference-systems). Above, we can see that the `world` object has the WGS84 ellipsoid, uses the Greenwich prime meridian, and the latitude and longitude axis order. We also have the suitable suitable area specification for the use of this CRS, and CRS identifier: `EPSG:4326`.

The CRS specification object, such as `world.crs`, has several other useful properties and methods to retrieve additional information about the used CRS. For example, try to run:

* `world.crs.is_geographic` to check is the CRS is geographic or not
* `world.crs.axis_info[0].unit_name` and `world.crs.axis_info[1].unit_name` to find out the CRS units of both axes (typically having identical units)
* `world.crs.to_authority()` extracts the authority (e.g., `EPSG`) and the identifier (e.g., `4326`)
* `world.crs.to_proj4()` returns the proj-string representation

In cases when a coordinate reference system (CRS) is missing or the wrong CRS is set, the `.set_crs` method can be used on a `GeoSeries` or a `GeoDataFrame` to set it. The CRS can be specified using an EPSG code as the first argument. In case the object already has a different CRS definition, we must also specify `allow_override=True` to replace it (otherwise we get an error). For example, here we set the `EPSG:4326` CRS, which has no effect because `world` already has that exact CRS definition:


In [ ]:
world2 = world.set_crs(4326)

and here we replace the definition from the existing `EPSG:4326`, to a new definition `EPSG:3857`:


In [ ]:
world3 = world.set_crs(3857, allow_override=True)

A number is interpreted as an `EPSG` code. We can also use strings, as in `'EPSG:4326'`, which is useful to make the code more clear and when using other authorities:


In [ ]:
world4 = world.set_crs('ESRI:54009', allow_override=True)

In `rasterio`, the CRS information is stored as part of a raster file connection metadata (@sec-using-rasterio). Replacing the CRS definition for a `rasterio` file connection is typically not necessary, because it is not considered in any operation, only the transformation matrix and coordinates are. One exception is when writing the raster, in which case we need to construct the metadata of the raster file to be written, and therein specify the CRS anyway (@sec-raster-from-scratch). However, if we do for some reason need to change the CRS definition in the file connection metadata, we can do that when opening the file in `r+` (reading and writing) mode:


In [ ]:
src_nlcd2 = rasterio.open('data/nlcd.tif', 'r+')
src_nlcd2.crs

In [ ]:
src_nlcd2.crs = 3857
src_nlcd2.crs

Importantly, the `.set_crs` (for vector layers) or the assignment to `.crs` (for rasters), as shown above, do not alter coordinates' values or geometries. Their role is only to set a metadata information about the object CRS. Consequently, the objects we created, `world3`, `world4`, and `src_nlcd2` are "incorrect", in the sense that the geometries are in fact given in a different CRS than specified in the associated CRS definition.

In some cases the CRS of a geographic object is unknown, as is the case in the london dataset created in the code chunk below, building on the example of London introduced in @sec-vector-layer-from-scratch:


In [ ]:
lnd_point = shapely.geometry.Point(-0.1, 51.5)
lnd_geom = gpd.GeoSeries([lnd_point])
lnd_layer = gpd.GeoDataFrame({'geometry': lnd_geom})
lnd_layer

In [ ]:
lnd_layer.crs

Nothing is printed as a result of the last expression, because the value of the CRS definition in `.crs` is `None`. This implies that `geopandas` does not know what the CRS is and is unwilling to guess. Unless a CRS is manually specified or is loaded from a source that has CRS metadata, `geopandas` does not make any explicit assumptions about which coordinate systems, other than to say "I don't know". This behavior makes sense given the diversity of available CRSs but differs from some approaches, such as the GeoJSON file format specification, which makes the simplifying assumption that all coordinates have a lon/lat CRS: `EPSG:4326`.

A CRS can be added to `GeoSeries` or `GeoDataFrame` objects using the `.set_crs` method, as mentioned above. Since the definition is missing, we do not need to specify `allow_override=True`, as there is nothing to override. For example:


In [ ]:
lnd_layer = lnd_layer.set_crs(4326)
lnd_layer.crs

In general, all geographic coordinates have a coordinate system and software can only make good decisions around plotting and and geometry operations if it knows what type of CRS it is working with. When working with `geopandas` and `rasterio`, datasets without a specified CRS are not an issue in most workflows, since only the coordinates are considered. It is up to the user to make sure that, when working with more than one layer, all of the coordinates are given in the same CRS (whether specified or not). When exporting the results, though, it is important to keep the CRS definition in place, because other software typically *do* use, and require, the CRS definition in calculation. It should also be mentioned that, in some cases the CRS specification is left unspecified on purpose, for example when working with layers in arbitrary or non-geographic space (simulations, internal building plans, analysis of plot-scale ecological patterns, etc.). 

## Geometry operations on projected and unprojected data {#sec-geometry-operations-on-projected-and-unprojected-data}

The `geopandas` package, through its dependency `shapely`, assumes planar geometry and works with distance/area values assumed to be in CRS units. 
In fact, the CRS definition is typically ignored, and the respective functions (such as in plotting and distance calculations) are applied on the "bare" `shapely` geometries. 
Accordingly, it is crucial to make sure that: 

* Geometric calculations are only applied in projected CRS
* If there is more than one layer involved---all layers have to be in the same (projected) CRS
* The input distance and area values are passed in CRS units (and the returned values---interpreted in CRS units)

For example, to calculate a buffer of 100 $km$ around London, we need to:

* Work with a layer representing London in a projected CRS (e.g., `EPSG:27700`)
* Pass the distance value in the CRS units (e.g., `100000` $m$)

In the following code section we create, from scratch, a point layer `lnd_layer_proj` with a point representing London (compare to `lnd_layer`, in a geographical CRS which we created above, see @sec-querying-and-setting-coordinate-systems):


In [ ]:
lnd_point_proj = shapely.geometry.Point(530000, 180000)
lnd_geom_proj = gpd.GeoSeries([lnd_point_proj], crs=27700)
lnd_layer_proj = gpd.GeoDataFrame({'geometry': lnd_geom_proj})
lnd_layer_proj

Now we can use the `.buffer` method (@sec-buffers) to calculate the buffer:


In [ ]:
lnd_layer_proj_buff = lnd_layer_proj.buffer(100000)
lnd_layer_proj_buff

The resulting buffer is shown in the left panel of @fig-reprojection-geo-proj.

Calculating a 100-$km$ buffer `lnd_layer`, which is in a geographical CRS, is impossible. 
Since the `lnd_layer` is in decimal degrees, the closest thing to a 100-$km$ buffer would be to use a distance of 1 degree, which is roughly equivalent to 100 $km$ (1 degree is about 111 $km$ at the equator): 


In [ ]:
lnd_layer_buff = lnd_layer.buffer(1)
lnd_layer_buff

However, this is incorrect, as told by the warning message and shown in the right panel of @fig-reprojection-geo-proj. 
The association between degrees and true distance varies over the surface of the earth and we cannot assume it is fixed (such as 1 degree = 111 $km$).


In [ ]:
#| label: fig-reprojection-geo-proj
#| fig-cap: 'Buffers around London, around a projected point and distance of 100 $km$ (left), and around a point in lon/lat using distance of 1 degree (right) which is incorrect.'

uk = world[world['name_long'] == 'United Kingdom']
uk_proj = uk.to_crs(27700)
fig, axes = plt.subplots(1, 2, figsize=(8, 4))
uk_proj.plot(color='none', edgecolor='darkgrey', ax=axes[0])
lnd_layer_proj.plot(color='red', ax=axes[0])
lnd_layer_proj_buff.plot(color='none', ax=axes[0])
uk.plot(color='none', edgecolor='darkgrey', ax=axes[1])
lnd_layer.plot(color='red', ax=axes[1])
lnd_layer_buff.plot(color='none', ax=axes[1])
axes[0].set_title('100 km buffer\ncorrect')
axes[1].set_title('1 degree buffer\nincorrectly approximating 100 km');

::: {.callout-note}
The distance between two lines of longitude, called meridians, is around 111 $km$ at the equator (execute `import geopy.distance;geopy.distance.geodesic((0,0),(0,1))` to find the precise distance). 
This shrinks to zero at the poles. 
At the latitude of London, for example, meridians are less than 70 $km$ apart (challenge: execute code that verifies this). 
Lines of latitude, by contrast, are equidistant from each other irrespective of latitude: they are always around 111 $km$ apart, including at the equator and near the poles (see Figure...). 
:::

::: {.callout-note}
The [`spherely`](https://github.com/benbovy/spherely) package, which is in early stages of development, is aimed at providing a spherical-geometry counterpart to `shapely`, so that true distances (in $m$) and areas (in $m^2$) can be calculated on geometries in geographic CRS.
:::

## When to reproject? {#sec-when-to-reproject}

The previous section showed how to set the CRS manually, with `lnd_layer.set_crs(4326)`. 
In real world applications, however, CRSs are usually set automatically when data is read-in. 
In many projects the main CRS-related task is to transform objects, from one CRS into another. But when should data be transformed? And into which CRS? There are no clear-cut answers to these questions and CRS selection always involves trade-offs (Maling 1992, add reference...). 
However, there are some general principles provided in this section that can help you decide.

First it's worth considering when to transform. In some cases transformation to a geographic CRS is essential, such as when publishing data online (for example, a Leaflet-based map using Python package `folium`). 
Another case is when two objects with different CRSs must be compared or combined, as shown when we try to find the distance between two objects with different CRSs:


In [ ]:
lnd_layer.distance(lnd_layer_proj)

To make the `lnd_layer` and `lnd_layer_proj` objects geographically comparable one of them must be transformed into the CRS of the other. 
But which CRS to use? 
The answer depends on context: many projects, especially those involving web mapping, require outputs in `EPSG:4326`, in which case it is worth transforming the projected object. 
If, however, the project requires geometric calculations, implying planar geometry, e.g., to calculating buffers (@sec-geometry-operations-on-projected-and-unprojected-data), it is necessary to transform data with a geographic CRS into an equivalent object with a projected CRS, such as the British National Grid (`EPSG:27700`). 
That is the subject of @sec-reprojecting-raster-geometries.

## Which CRS to use? {#sec-which-crs-to-use}

The question of which CRS is tricky, and there is rarely a "right" answer: "There exist no all-purpose projections, all involve distortion when far from the center of the specified frame" (Bivand, Pebesma, and Gómez-Rubio 2013, add citation...). 
Additionally, you should not be attached just to one projection for every task. 
It is possible to use one projection for some part of the analysis, another projection for a different part, and even some other for visualization. 
Always try to pick the CRS that serves your goal best!

When selecting **geographic** CRSs, the answer is often [WGS84](https://en.wikipedia.org/wiki/World_Geodetic_System#A_new_World_Geodetic_System:_WGS_84). 
It is used not only for web mapping, but also because GPS datasets and thousands of raster and vector datasets are provided in this CRS by default. 
WGS84 is the most common CRS in the world, so it is worth knowing its EPSG code: 4326. 
This "magic number" can be used to convert objects with unusual projected CRSs into something that is widely understood.

What about when a **projected** CRS is required? 
In some cases, it is not something that we are free to decide: "often the choice of projection is made by a public mapping agency" (Bivand, Pebesma, and Gómez-Rubio 2013, add citation...). 
This means that when working with local data sources, it is likely preferable to work with the CRS in which the data was provided, to ensure compatibility, even if the official CRS is not the most accurate. 
The example of London was easy to answer because: 

* the British National Grid (with its associated EPSG code 27700) is well known, and 
* the original dataset (`lnd_layer`) already had that CRS.

A commonly used default is Universal Transverse Mercator ([UTM](https://en.wikipedia.org/wiki/Universal_Transverse_Mercator_coordinate_system)), a set of CRSs that divides the Earth into 60 longitudinal wedges and 20 latitudinal segments. 
The transverse Mercator projection used by UTM CRSs is conformal but distorts areas and distances with increasing severity with distance from the center of the UTM zone. 
Documentation from the GIS software Manifold therefore suggests restricting the longitudinal extent of projects using UTM zones to 6 degrees from the central meridian (source: [manifold.net](http://www.manifold.net/doc/mfd9/universal_transverse_mercator_projection.htm)). 
Therefore, we recommend using UTM only when your focus is on preserving angles for relatively small area!

Almost every place on Earth has a UTM code, such as "60H" which refers to northern New Zealand. 
UTM EPSG codes run sequentially from 32601 to 32660 for northern hemisphere locations and from 32701 to 32760 for southern hemisphere locations.

To show how the system works, let's create a function, `lonlat2UTM` to calculate the EPSG code associated with any point on the planet as follows:


In [ ]:
import math
def lonlat2UTM(lon, lat):
    utm = (math.floor((lon + 180) / 6) % 60) + 1
    if lat > 0:
        utm += 32600
    else:
        utm += 32700
    return utm

The following command uses this function to identify the UTM zone and associated EPSG code for Auckland:


In [ ]:
lonlat2UTM(174.7, -36.9)

Here is another example for London (where we "unpack" the coordinates of the 1^st^ geometry in `lnd_layer` into the `lonlat2UTM` function arguments):


In [ ]:
lonlat2UTM(*lnd_layer['geometry'].iloc[0].coords[0])

Currently, we also have tools helping us to select a proper CRS. 
For example, the webpage https://jjimenezshaw.github.io/crs-explorer/ lists CRSs based on selected location and type. 
Important note: while these tools are helpful in many situations, you need to be aware of the properties of the recommended CRS before you apply it.

In cases where an appropriate CRS is not immediately clear, the choice of CRS should depend on the properties that are most important to preserve in the subsequent maps and analysis. 
All CRSs are either equal-area, equidistant, conformal (with shapes remaining unchanged), or some combination of compromises of those (@sec-projected-coordinate-reference-systems). 
Custom CRSs with local parameters can be created for a region of interest and multiple CRSs can be used in projects when no single CRS suits all tasks. 
"Geodesic calculations" can provide a fall-back if no CRSs are appropriate (see proj.org/geodesic.html). 
Regardless of the projected CRS used, the results may not be accurate for geometries covering hundreds of kilometers.

When deciding on a custom CRS, we recommend the following:

* A Lambert azimuthal equal-area ([LAEA](https://en.wikipedia.org/wiki/Lambert_azimuthal_equal-area_projection)) projection for a custom local projection (set latitude and longitude of origin to the center of the study area), which is an equal-area projection at all locations but distorts shapes beyond thousands of kilometers

* Azimuthal equidistant ([AEQD](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection)) projections for a specifically accurate straight-line distance between a point and the center point of the local projection

* Lambert conformal conic ([LCC](https://en.wikipedia.org/wiki/Lambert_conformal_conic_projection)) projections for regions covering thousands of kilometers, with the cone set to keep distance and area properties reasonable between the secant lines

* Stereographic ([STERE](https://en.wikipedia.org/wiki/Stereographic_projection)) projections for polar regions, but taking care not to rely on area and distance calculations thousands of kilometers from the center

One possible approach to automatically select a projected CRS specific to a local dataset is to create an azimuthal equidistant (AEQD) projection for the center-point of the study area. 
This involves creating a custom CRS (with no EPSG code) with units of meters based on the center point of a dataset. 
Note that this approach should be used with caution: no other datasets will be compatible with the custom CRS created and results may not be accurate when used on extensive datasets covering hundreds of kilometers.

The principles outlined in this section apply equally to vector and raster datasets. 
Some features of CRS transformation however are unique to each geographic data model. 
We will cover the particularities of vector data transformation in @sec-reprojecting-vector-geometries and those of raster transformation in @sec-reprojecting-raster-geometries. 
Next, the last section, shows how to create custom map projections (@sec-custom-map-projections).

## Reprojecting vector geometries {#sec-reprojecting-vector-geometries}

@sec-spatial-class demonstrated how vector geometries are made-up of points, and how points form the basis of more complex objects such as lines and polygons. Reprojecting vectors thus consists of transforming the coordinates of these points, which form the vertices of lines and polygons.

@sec-geometry-operations-on-projected-and-unprojected-data contains an example in which at least one `GeoDataFrame` object must be transformed into an equivalent object with a different CRS to calculate the distance between two objects (?).

...

## Reprojecting raster geometries {#sec-reprojecting-raster-geometries}

The projection concepts described in the previous section apply equally to rasters. 
However, there are important differences in reprojection of vectors and rasters: transforming a vector object involves changing the coordinates of every vertex but this does not apply to raster data. 
Rasters are composed of rectangular cells of the same size (expressed by map units, such as degrees or meters), so it is usually impracticable to transform coordinates of pixels separately. 
Raster reprojection involves creating a new raster object, often with a different number of columns and rows than the original. 
The attributes must subsequently be re-estimated, allowing the new pixels to be 'filled' with appropriate values. 
In other words, raster reprojection can be thought of as two separate spatial operations: a vector reprojection of the raster extent to another CRS (@sec-reprojecting-vector-geometries), and computation of new pixel values through resampling (@sec-raster-resampling). 
Thus in most cases when both raster and vector data are used, it is better to avoid reprojecting rasters and reproject vectors instead.

:::{.callout-note}
Reprojection of the regular rasters is also known as warping. 
Additionally, there is a second similar operation called "transformation". 
Instead of resampling all of the values, it leaves all values intact but recomputes new coordinates for every raster cell, changing the grid geometry. 
For example, it could convert the input raster (a regular grid) into a curvilinear grid. The `rasterio`, like common raster file formats (such as GeoTIFF), does not support curvilinear grids (?).
:::

The raster reprojection process is done using two functions from the `rasterio.warp` sub-package:

* `rasterio.warp.calculate_default_transform`
* `rasterio.warp.reproject`

The first function, `calculate_default_transform`, is used to calculate the new transformation matrix in the destination CRS, according to the source raster dimensions and bounds. 
Alternatively, the destination transformation matrix can be obtained from an existing raster; this is common practice when we need to align one raster with another, for instance to be able to combine them in raster algebra operations (@sec-raster-local-operations) (see below). 
The second function `rasterio.warp.reproject` then actually calculates cell values in the destination grid, using the user-selected resampling method (such as nearest neighbor, or bilinear). 

Let's take a look at two examples of raster transformation: using categorical and continuous data. 
Land cover data are usually represented by categorical maps. The `nlcd.tif` file provides information for a small area in Utah, USA obtained from National Land Cover Database 2011 in the NAD83 / UTM zone 12N CRS, as shown in the output of the code chunk below (only first line of output shown). 
We already created a connection to the `nlcd.tif` file, named `src_nlcd`:


In [ ]:
src_nlcd

Recall that the raster transformation matrix and dimensions are accessible from the file connection as follows. This information will be required to calculate the destination transformation matrix (hereby printed collectively in a `tuple`):


In [ ]:
src_nlcd.transform, src_nlcd.width, src_nlcd.height

First, let's define the destination CRS. In this case, we choose WGS84 (EPSG code `4326`):


In [ ]:
dst_crs = 'EPSG:4326'

Now, we are ready to claculate the destination raster transformation matrix (`dst_transform`), and the destination dimensions (`dst_width`, `dst_height`), as follows: 


In [ ]:
dst_transform, dst_width, dst_height = rasterio.warp.calculate_default_transform(
    src_nlcd.crs,
    dst_crs,
    src_nlcd.width,
    src_nlcd.height,
    *src_nlcd.bounds
)
dst_transform, dst_width, dst_height

Note that `*`, in `*src_nlcd.bounds`, is used to unpack `src_nlcd.bounds` to four separate arguments, which `calculate_default_transform` requires:


In [ ]:
src_nlcd.bounds

Next, we will create the metadata file used for writing the reprojected raster to file. 
For convenience, we are taking the metadata of the source raster (`src_nlcd.meta`), making a copy (`dst_kwargs`), and then updating those specific properties that need to be changed. 
Note that the reprojection process typically creates "No Data" pixels, even when there were none in the input raster, since the raster orientation changes and the edges need to be "filled" to get back a rectangular extent. 
We need to specify a "No Data" value of our choice, if there is none, or use the existing source raster setting, such as `255` in this case:


In [ ]:
dst_kwargs = src_nlcd.meta.copy()
dst_kwargs.update({
    'crs': dst_crs,
    'transform': dst_transform,
    'width': dst_width,
    'height': dst_height
})
dst_kwargs

We are ready to create the reprojected raster. 
Here, reprojection takes place between two file connections, meaning that the raster value arrays are not being read into memory at once. 
It is also possible to reproject into an in-memory `ndarray` object, see the [documentation](https://rasterio.readthedocs.io/en/latest/api/rasterio.warp.html#rasterio.warp.reproject).

To write the reprojected raster, we first create a destination file connection `dst_nlcd`, pointing at the output file path of our choice (`output/nlcd_4326.tif`), using the updated metadata object created earlier (`dst_kwargs`):


In [ ]:
dst_nlcd = rasterio.open('output/nlcd_4326.tif', 'w', **dst_kwargs)

Then, we use the `rasterio.warp.reproject` function to calculate and write the reprojection result into the `dst_nlcd` file connection. 
Note that the `source` and `destination` accept a "band" object, created using `rasterio.band`. 
In this case, there is just one band. 
If there were more bands, we would have to repeat the procedure for each band, using `i` instead of `1` inside a [loop](https://rasterio.readthedocs.io/en/latest/topics/reproject.html#reprojecting-a-geotiff-dataset):


In [ ]:
rasterio.warp.reproject(
    source=rasterio.band(src_nlcd, 1),
    destination=rasterio.band(dst_nlcd, 1),
    src_transform=src_nlcd.transform,
    src_crs=src_nlcd.crs,
    dst_transform=dst_transform,
    dst_crs=dst_crs,
    resampling=rasterio.enums.Resampling.nearest
)

Finally, we close the file connection so that the data are actually written:


In [ ]:
dst_nlcd.close()

Many properties of the new object differ from the previous one, including the number of columns and rows (and therefore number of cells), resolution (transformed from meters into degrees), and extent, as summarized again below (note that the number of categories increases from 8 to 9 because of the addition of NA values, not because a new category has been created---the land cover classes are preserved).


In [ ]:
src_nlcd.meta

In [ ]:
src_nlcd_4326 = rasterio.open('output/nlcd_4326.tif')
src_nlcd_4326.meta

Examining the unique raster values tells us that the new raster has the same categories, plus the value `255` representing "No Data":


In [ ]:
np.unique(src_nlcd.read(1))

In [ ]:
np.unique(src_nlcd_4326.read(1))

In [ ]:
#| label: fig-raster-reproject-nlcd
#| fig-cap: Reprojecting a categorical raster using nearest neighbor resampling

fig, axes = plt.subplots(ncols=2, figsize=(8,4))
show(src_nlcd, ax=axes[0], cmap='Set3')
show(src_nlcd_4326, ax=axes[1], cmap='Set3')
axes[0].set_title('Original (EPSG:26912)')
axes[1].set_title('Reprojected (EPSG:4326)');

In the above example, we automatically calculated an optimal (i.e., most information preserving) destination grid using `rasterio.warp.calculate_default_transform`. 
This is appropriate when there are no specific requirements for the destination raster spatial properties. 
Namely, we are not required to otain a specific origin and resolution, but just wish to preserve the raster values as much as possible.
To do that, `calculate_default_transform` "tries" to keep the extent and resolution of the destination raster as similar as possible to the source. 
In other situations, however, we need to reproject a raster into a specific "template", so that it corresponds, for instance, with other rasters we use in the analysis. 
In the following code section, we reproject the `nlcd.tif` raster, again, buit this time using the `nlcd_4326.tif` reprojection result as the "template" to demonstrate this alternative workflow.

First, we create a connection to our "template" raster to read its metadata:


In [ ]:
template = rasterio.open('output/nlcd_4326.tif')
template.meta

Then, we create a write-mode connection to our destination raster, using this metadata, meaning that as the resampling result is going to have identical metadata as the "template":


In [ ]:
dst_nlcd_2 = rasterio.open('output/nlcd_4326_2.tif', 'w', **template.meta)

Now, we can resample and write the result:


In [ ]:
rasterio.warp.reproject(
    source=rasterio.band(src_nlcd, 1),
    destination=rasterio.band(dst_nlcd_2, 1),
    src_transform=src_nlcd.transform,
    src_crs=src_nlcd.crs,
    dst_transform=dst_nlcd_2.transform,
    dst_crs=dst_nlcd_2.crs,
    resampling=rasterio.enums.Resampling.nearest
)

In [ ]:
dst_nlcd_2.close()

Naturally, in this case, the outputs `nlcd_4326.tif` and `nlcd_4326_2.tif` are identical, as we used the same "template" and the same source data: 


In [ ]:
d = rasterio.open('output/nlcd_4326.tif').read(1) == rasterio.open('output/nlcd_4326_2.tif').read(1)
d

In [ ]:
np.all(d)

The difference is that in the first example we calculate the template automatically, using `rasterio.warp.calculate_default_transform`, while in the second example we used an existing raster as the "template".

Importantly, when the template raster has much more "coarse" resolution than the source raster, the:

* `rasterio.enums.Resampling.average` (for continuous rasters), or 
* `rasterio.enums.Resampling.mode` (for categorical rasters)

resampling method should be used, instead of `rasterio.enums.Resampling.nearest`. 
Otherwise, much of the data will be lost, as the "nearest" method can capture one pixel value only for each destination raster pixel. 

Reprojecting continuous rasters (with numeric or, in this case, integer values) follows an almost identical procedure. 
This is demonstrated below with `srtm.tif` from the Shuttle Radar Topography Mission (SRTM), which represents height in meters above sea level (elevation) with the WGS84 CRS.

We will reproject this dataset into a projected CRS, but not with the nearest neighbor method which is appropriate for categorical data. 
Instead, we will use the bilinear method which computes the output cell value based on the four nearest cells in the original raster. 
The values in the projected dataset are the distance-weighted average of the values from these four cells: the closer the input cell is to the center of the output cell, the greater its weight. 
The following code section create a text string representing WGS 84 / UTM zone 12N, and reproject the raster into this CRS, using the bilinear method. 
The code is practically the same, except for changing the source and destination file names, and replacing `nearest` with `bilinear`:


In [ ]:
dst_crs = 'EPSG:32612'
dst_transform, dst_width, dst_height = rasterio.warp.calculate_default_transform(
    src_srtm.crs,
    dst_crs,
    src_srtm.width,
    src_srtm.height,
    *src_srtm.bounds
)
dst_kwargs = src_srtm.meta.copy()
dst_kwargs.update({
    'crs': dst_crs,
    'transform': dst_transform,
    'width': dst_width,
    'height': dst_height
})
dst_srtm = rasterio.open('output/srtm_32612.tif', 'w', **dst_kwargs)
rasterio.warp.reproject(
    source=rasterio.band(src_srtm, 1),
    destination=rasterio.band(dst_srtm, 1),
    src_transform=src_srtm.transform,
    src_crs=src_srtm.crs,
    dst_transform=dst_transform,
    dst_crs=dst_crs,
    resampling=rasterio.enums.Resampling.bilinear
)
dst_srtm.close()

@fig-raster-reproject-srtm shows the input and the reprojected SRTM rasters.


In [ ]:
#| label: fig-raster-reproject-srtm
#| fig-cap: Reprojecting a continuous raster using bilinear resampling

fig, axes = plt.subplots(ncols=2, figsize=(8,4))
show(src_srtm, ax=axes[0])
show(rasterio.open('output/srtm_32612.tif'), ax=axes[1])
axes[0].set_title('Original (EPSG:4326)')
axes[1].set_title('Reprojected (EPSG:32612)');

## Custom map projections {#sec-custom-map-projections}

...

## Exercises
